In [1]:
# Processing and Displaying run results
import os
import pandas as pd

In [2]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [3]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%)


In [4]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)


In [5]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [6]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [7]:
from tabulate import tabulate

In [8]:
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))

KeyError: "['sgd', 'nonaccelgamsgd', 'gamsgd'] not in index"

### Gradient Norm Only Minimization (GNOM)

#### GNOM learning rate testing (20 epochs):

In [ ]:
# Specify the directory
folder_path_gnom = 'logs/GNOM-Tests'  


In [9]:
def extract_epoch_time(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    epoch_time = 0
    count = 0
    for line in lines:
        if 'Epoch' and 'time:' in line:
            count += 1
            epoch_time += float(line.strip().split(': ')[1].split(' ')[0])
    return (epoch_time / count)

def extract_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    epochs = 20
    if len(parts) >= 6:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size = None, None, None, None, None, None
    
    if len(parts) == 7:
        learning_rate = parts[5]
        batch_size = parts[6].split('.')[0] 
        if batch_size == 'full':
            batch_size = 128
            epochs = 200
    else:
        learning_rate = parts[5][:-4]
        batch_size = 128
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

In [10]:
gnom_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if '-lr-' in file:  
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        epoch_time = extract_epoch_time(file_path)
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_test_info(file_path)
        
        gnom_df = gnom_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Learning Rate': float(learning_rate),
            'Batch Size': int(batch_size),
            'Epochs': epochs,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time,
            'Avg Epoch Time (s)': epoch_time
        }, ignore_index=True)

gnom_df['Total Time (min)'] = gnom_df['Total Time (s)'] / 60

gnom_df_sorted = gnom_df.sort_values(by=['Batch Size', 'Learning Rate'])

gnom_df_sorted

NameError: name 'folder_path_gnom' is not defined

#### GNOM Gradient Norm and Hessian Eigenvalue testing:

In [12]:
def extract_ev_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) == 9:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7]
    elif len(parts) == 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7][:-4]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = None, None, None, None, None, None, None
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

def extract_grad_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    grad_norm = None
    top_ev = None
    smallest_ev = None
    for line in reversed(lines):
        if 'Smallest Hessian Eigenvalue:' in line:
            smallest_ev = float(line.strip().split(': ')[1])
        if 'Norm of the Gradient:' in line:
            grad_norm = float(line.strip().split(': ')[1])
        if 'Largest Hessian Eigenvalue:' in line:
            top_ev = float(line.strip().split(': ')[1])
            break

    return grad_norm, top_ev, smallest_ev

In [14]:
folder_path_gnom_ev = 'logs/GNOM-Saved-Models'  
gnom_ev_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_ev):
    file_path = os.path.join(folder_path_gnom_ev, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_ev_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    
    gnom_ev_df = gnom_ev_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev
    }, ignore_index=True)

gnom_ev_df['Total Time (min)'] = gnom_ev_df['Total Time (s)'] / 60
gnom_ev_df['Avg Epoch Time (min)'] = gnom_ev_df['Avg Epoch Time (s)'] / 60
gnom_ev_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_ev_df_sorted = gnom_ev_df.sort_values(by=['Epochs','Optimizer','Batch Size'])

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")
gnom_ev_df_sorted


All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations


/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_86718/1462318495.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_ev_df = gnom_ev_df._append({


,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Smallest Eigenvalue of Hessian,Total Time (min),Avg Epoch Time (min)
5,gam,0.10,128,200,96.41,0.396285,45.6671,-2.5961,332.266464,3.306122
7,gnomsgd,0.05,128,200,10.00,0.034401,0.1001,-0.0973,396.257328,3.942846
0,gnomsgd,0.01,256,200,11.14,0.066849,0.5386,-0.1408,372.538592,3.706831
3,gnomsgd,0.01,512,200,16.97,0.111012,1.0676,-0.1032,354.544338,3.527792
4,nonaccelgam,0.10,128,200,95.59,0.273106,59.2035,-1.7100,786.008842,7.820976
11,sgd,0.10,128,200,95.50,0.057379,25.0349,-0.1720,83.996313,0.835771
10,gam,0.10,128,50,94.52,0.918708,32.4005,-2.5729,82.848856,3.248966
9,gnomsgd,0.05,128,50,10.00,0.062882,0.0998,-0.0332,99.930228,3.918811
2,gnomsgd,0.01,256,50,12.71,0.070527,0.1035,-0.0133,93.470016,3.665468
8,gnomsgd,0.01,512,50,14.24,0.158222,1.0770,-0.0801,88.931464,3.487495
